# Import packages

In [4]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import youdao


def mk_dir(file_path):

    folder = os.path.exists(file_path)
    if not folder:
        os.makedirs(file_path)

# [Mesh]词的搜索结果,筛选Pubtator结果

In [41]:
keyword = json.load(open("../3.KeyWord/keyword.json", "r"))
input_path = "../2.PubTator/output"
sub = [
    "BACKGROUND:", "BACKGROUND/AIM:", "METHOD:", "METHODS:", "FINGDING",
    "FINDINGS:", "RESULT", "RESULTS:", "CONCLUSION", "CONCLUSIONS:",
    "OBJECTIVE:", "OBJECTITVES"
]

# 建立物种大类列表6

sp_l = [ele for ele in os.listdir(input_path) if "_done" not in ele]
sp_l = [ele for ele in sp_l if ".ipynb" not in ele]

b = []
paper_ad = []
for sp in sp_l:

    paper_d = {}
    paper_pd = {}
    paper_ld = {}
    paper_md = {}
    paper_pld = {}
    paper_pmd = {}
    mk_dir("paper_md/%s" % sp)
    mk_dir("paper_ld/%s" % sp)

    # 建立这个大类中物种列表
    print(sp)
    spec_l = [
        ele for ele in os.listdir("%s/%s" % (input_path, sp))
        if ".ipynb" not in ele
    ]

    for spec in spec_l:

        spec_d = {}  # 用于储存paper
        spec_pd = {}
        file_l = [
            ele for ele in os.listdir("%s/%s/%s" % (input_path, sp, spec))
            if ".json" in ele
        ]  # 建立这个物种的三种搜索结果json文件列表

        for file in file_l:

            # 读取json文件中的paper并建立列表
            input_name = "%s/%s/%s/%s" % (input_path, sp, spec, file)
            data_l = json.load(open("%s" % input_name, 'r'))

            for paper in data_l:

                # 读取pubtator的注释
                title = paper["passages"][0]["text"]
                ab = paper["passages"][1]["text"]
                screen_l = keyword["LungRelated"]
                key_l = [
                    word for word in keyword["LungRelated"]
                    if word in title + ab
                ]

                if len(key_l) > 0:  # 筛选3

                    full = "SPECIES:%s\nID:%s\nTITLE:%s\nABSTRACT:%s" % (
                        spec.replace("_", " "), paper["id"], title, ab)

                    # read title annotation
                    title_anno = paper["passages"][0]["annotations"]
                    title_disease_l = [
                        i['text'] for i in title_anno
                        if i['infons']['type'] == 'Disease'
                    ]
                    title_species_l = [
                        i['text'] for i in title_anno
                        if i['infons']['type'] == 'Species'
                    ]

                    ab_anno = paper["passages"][1]["annotations"]
                    full_disease_l = title_disease_l + [
                        i['text']
                        for i in ab_anno if i['infons']['type'] == 'Disease'
                    ]
                    full_species_l = title_species_l + [
                        i['text']
                        for i in ab_anno if i['infons']['type'] == 'Species'
                    ]
                    pubtator_d = {}

                    pubtator_d["dis"] = title_disease_l + full_disease_l
                    #                     pubtator_d["spec"] = title_species_l + full_species_l
                    pubtator_d["spec"] = spec.replace("_", " ")

                    for word in sub:
                        if word in full:
                            full = full.replace(word, "\n%s" % word)
                            full = full.replace("\n\n", "\n")

                    spec_d[paper["id"]] = full
                    spec_pd[paper["id"]] = pubtator_d

        paper_d[spec] = spec_d
        paper_pd[spec] = spec_pd
    paper_ad.append(paper_d)
    a = []
    for specc in paper_d:
        if len(paper_d[specc]) >= 10:
            if len(paper_d[specc]) > 0:
                for paperr in paper_d[specc]:
                    a.append([int(paperr), specc.replace("_", " ")])
                    with open(
                            "%s/%s/%s.txt" %
                        ("paper_ld", sp, specc + "_" + paperr), "w") as f:
                        f.write(paper_d[specc][paperr])
    b.append(a)

Anaerobic
Curved
GramN
GramPc
GramPr
Intracellular


## Add species which are overwritting by same paper

In [ ]:
c = []
for sp in sp_l:
    d = pd.read_excel("文献标注—校对/%s.xls" % sp, index_col=0, header=0)
    d["ID"] = d.index
    d = d.loc[:, ["ID", "物种名称"]]
    d = d.values.tolist()
    c.append(d)

# f = []
# for i  in range(len(b)):
#     e = []
#     for j in range(len(b[i])):
#         if b[i][j] not in c[i]:
#             print(b[i][j])
#             e.append(b[i][j])
#     print(len(c[i]), len(b[i]))
#     print(len(e))
#     f.append(e)

# for i in range(len(sp_l)):
#     d = pd.read_excel("文献标注—校对/%s.xls" % sp_l[i], index_col=0, header=0)
#     d["ID"] = d.index
#     g = pd.DataFrame(f[i], columns = ["ID", "物种名称"] )
#     print(g.shape)
#     df = pd.concat([d, g], axis=0)
#     for j in df.index:
#         df.loc[j, "id"] = "https://pubmed.ncbi.nlm.nih.gov/%s/" % df.loc[j, "ID"]
#     df.to_excel("%s.xls" % (sp_l[i] + "_add"))

# 添加翻译（真贵！！！）

In [5]:
input_path = "XY"
sped_l = os.listdir(input_path)
keyword = json.load(open("../3.KeyWord/keyword.json", "r"))

for sped in sped_l:

    mk_dir("paper_ld/%s/input" % sped)

    paper_l = os.listdir("%s/%s/outputs" % (input_path, sped))
    paper_l = [ele for ele in paper_l if "ipynb" not in ele]

    for paper in paper_l:

        # 添加翻译
        folder = os.path.exists("paper_ldcn/%s/%s" % (sped, paper))

        if not folder:

            with open("%s/%s/%s" % (input_path, sped, paper), "r") as f:
                full = f.read()
                f.close()
            if len(full) > 5000:
                full_1 = full[0:5000]
                full_2 = full[5000:len(full)]

                full1_cn = youdao.youdao_trans(full_1)
                full2_cn = youdao.youdao_trans(full_2)
                full_t = full + "\n" + full1_cn + full2_cn
            else:
                full_cn = youdao.youdao_trans(full)
                full_t = full + "\n" + full_cn
            with open("paper_ldcn/%s/%s" % (sped, paper), "w") as f:
                f.write(full_t)
                f.close()

        with open(
                "%s/%s/%s_pubtator.json" %
            (input_path, sped, paper.replace(".json", "")), "r") as f:
            pubtator_d = json.load(f)
            dis_l = pubtator_d["dis"]
            rm_l = [
                "infection", "infections", "infect", "Infection", "Infections",
                "Infect"
            ]
            dis_l = [ele for ele in dis_l if ele not in rm_l]
            sp_l = pubtator_d["spec"]
            f.close()
        keyword["DISEASE"] = np.unique(dis_l)
        keyword["SPECIES"] = np.unique(sp_l)
        with open("%s/%s/outputs/%s" % (input_path, sped, paper), "r") as f:
            txt = json.load(f)
            full = txt['content']
            f.close()

        anot = {
            'type': 'T',
            'name': '',
            'value': '',
            'start': 0,
            'end': 0,
            'attributes': [],
            'id': 1
        }
        anoa = {'annotation': {'T': [''], 'E': [''], 'R': [''], 'A': ['']}}

        for pre_type in keyword.keys():
            anot["name"] = pre_type
            for word in keyword[pre_type]:
                if word in full:
                    anot["value"] = word
                    for m in re.finditer(
                            word.replace(")", " ").replace("(", " "),
                            full.replace("\n",
                                         "").replace(")",
                                                     " ").replace("(", " ")):
                        anot["start"] = m.start()
                        anot["end"] = m.end()
                        anot["id"] = len(anoa["annotation"]["T"])
                        guodu = deepcopy(anot)
                        anoa["annotation"]["T"].append(guodu)
        txt["outputs"] = anoa
        txt["labeled"] = "true"

#         with open("%s/%s/input/%s" % (input_path, sped, paper), "w") as f:
#             b = json.dumps(txt)
#             f.write(b)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'XY/GramN/outputs'

In [8]:
input_path = "XY"
paper_l = os.listdir(input_path)
import youdao

for paper in paper_l:

    # 添加翻译

    with open("%s/%s" % (input_path, paper), "r") as f:
        full = f.read()
        f.close()
    if len(full) > 5000:
        full_1 = full[0:5000]
        full_2 = full[5000:len(full)]

        full1_cn = youdao.youdao_trans(full_1)
        full2_cn = youdao.youdao_trans(full_2)
        full_t = full + "\n" + full1_cn + full2_cn
    else:
        full_cn = youdao.youdao_trans(full)
        full_t = full + "\n" + full_cn
    with open("XY_CN/%s" % (paper), "w") as f:
        f.write(full_t)
        f.close()


KeyError: 'translation'